Planejamento da Produção de Energia Elétrica 3

In [168]:
#                 Jan.  Feb.  Mar.  Apr.  May   Jun.  Jul.  Aug.  Sep.  Oct.  Nov.  Dec.
demands =        [2800, 2500, 2300, 2500, 2600, 2800, 2800, 2600, 2600, 2800, 3000, 3100] # in [MW * Month]
avg_flow_rate =  [[6375, 7741, 9107, 10473, 11839], # Jan.
                  [5382, 6535, 7688,  8841,  9995], # Feb.
                  [6550, 7954, 9358, 10762, 12165], # Mar.
                  [4756, 5775, 6794,  7814,  8832], # Apr.
                  [3012, 3658, 4303,  4948,  5594], # May
                  [2473, 3003, 3533,  4063,  4593], # Jun.
                  [2007, 2437, 2867,  3297,  3727], # Jul.
                  [1790, 2173, 2557,  2941,  3324], # Aug.
                  [1520, 1845, 2171,  2497,  2822], # Sep.
                  [1573, 1910, 2247,  2584,  2921], # Oct.
                  [2462, 2989, 3517,  4045,  4572], # Nov.
                  [2926, 3553, 4180,  4807,  5434]] # Dez.
                  # in [(m^3)/s]

avg_flow_rate_prob = [0.1, 0.2, 0.4, 0.2, 0.1]

# Hydro Power plant characteristics
MAX_PRODUCTION = 3230   # in [MW * Month]

# Reservoir characteristics
X_MAX   = 21200         # in [10^6 m^3]
X_MIN   = 12800         # in [10^6 m^3]
X_FINAL = 15000         # in [10^6 m^3]

DRENAGE_MAX = 8000     # in [(m^3)/s]
U_MAX       = 7955      # in [(m^3)/s]
U_MIN       = 1400      # in [(m^3)/s]

N_COEF  = 0.88
GRAVITY_ACC_COEF = 9.88 # in [m/s^2]

MONTHS_TO_CALCULATE = 12

In [169]:
# Generators

def U_k():
    step = 100   # u_k's vão de 10 em 10
    num = U_MIN
    while num < DRENAGE_MAX:
        yield num
        num += step     

In [170]:
#Funcoes auxiliares

def get_thermal_production_cost(g):
    return int(64.8 * (g**2))

# x is the reservoir volume in [10^6 m^3]
def h_1(x):
    return 303.04 + (0.15519e-2 * x) - (0.17377e-7 * x**2)

def h_2(u):
    return 279.84 + (0.22130e-3 * u)

def h(x,u):
    p = N_COEF * GRAVITY_ACC_COEF * 1e-3

    considered_u = u
    if u > U_MAX:
        considered_u = U_MAX
    elif u < U_MIN:
        considered_u = 0

    power_generariton = p * (h_1(x)-h_2(u)) * u
    return int(power_generariton) if power_generariton < MAX_PRODUCTION else MAX_PRODUCTION

def get_operation_cost(hydro_production, demand):
    if demand > hydro_production:
        return get_thermal_production_cost(demand - hydro_production)
    else:
        return 0

# fk(xk,uk)
def get_reservoir_volume(current_volume, water_out, water_in):
    return int(current_volume + (2.628 * (water_in - water_out)))


def get_volume_cost(volume):
    return int(0.0247 * (volume -12800) * 1e6)



In [172]:
def dp(month, current_reservoir_volume, costs_, choices_):
    if month == MONTHS_TO_CALCULATE:
        # Caso base (sem recursao)
        if current_reservoir_volume < X_FINAL:  # Condição final de factibiliade
            return -1                           # Estado infactivel (-1 representa infactibilidade) 
        else:
            return get_volume_cost(current_reservoir_volume)
    else:

        if (month, current_reservoir_volume) in costs_:         # verifica se esse estado ja não foi calculado e retorna
            return costs_[(month, current_reservoir_volume)]

        cost_min = 9999999999999999999999   # F_aux <- inf
        u_min    = 9999999999999999999999   # U_oaux <- inf

        for u_k in U_k():                   # for ∀uk ∈ Uk do

            cost = 0                        # Fxu_aux <- 0
            sable_state     = False # Variavel aux. que modela se esse u_k, dado todas w_k, possui algum estado factivel
            has_some_infeasable_state   = False # Variavel aux. que modela se esse u_k, dado todas w_k, possui algum estado infactivel
            has_some_feasable_state     = False # Variavel aux. que modela se esse u_k, dado todas w_k, possui algum estado factivel

            for index, potencial_flow in enumerate(avg_flow_rate[month]):


                probability = avg_flow_rate_prob[index]
                potencial_reservoir_volume = get_reservoir_volume(current_reservoir_volume, u_k, potencial_flow)

                    
                if X_MIN < potencial_reservoir_volume < X_MAX:    # Verifica factibilidade do estado x_k+1
                    onward_cost = dp(month + 1, potencial_reservoir_volume, costs_, choices_) # calculo de F(x_k+1)
                        
                    if onward_cost >= 0:
                        has_some_feasable_state = True
                        cost = probability * (get_operation_cost(h(current_reservoir_volume, u_k), demands[month]) + onward_cost) + cost

                # else:
                #     has_some_infeasable_state = True

            if cost < cost_min and has_some_feasable_state :
                # if month == 0:
                #     print(cost)
                cost_min = cost
                u_min = u_k

        if u_min == 9999999999999999999999:
            return -1
        else:
            costs_[(month, current_reservoir_volume)] = cost_min
            choices_[(month, current_reservoir_volume)] = u_min
            return cost_min

In [173]:
# Dicionarios para carregar custos otimos acumulados e escolhas otimas. Inves de ter um par de dicionarios por estado k, o estado k foi adicionado como uma das dimensões de X. Por Exemplo, a tupla (3, 16000) mapeia o estado X_3 = 16000
costs_      = {}
choices_    = {}

# Execução do algoritmo a partir do estado inicial
custo = dp(0, 15000, costs_, choices_)

print("Custo medio acumulado: ", custo)

print("Numero total de tomadas de decisão calculadas: ", len(choices_))


Custo medio acumulado:  22149.897628828756
Numero total de tomadas de decisão calculadas:  68648
